In [10]:
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
analyze_path = os.path.join(parent_dir, "utils")

os.chdir(analyze_path)

from new_grid import TaiwanBaseGridGenerator, AccidentHotspotAnalyzer

OSM_DIR = '/Users/wangqiqian/Desktop/ST-RTA-GIS/Data/road_new.shp/'
SHP_PATH = '../Data/OFiles_9e222fea-bafb-4436-9b17-10921abc6ef2/TOWN_MOI_1140318.shp'
ACCIDENT_PATH = '../ComputedDataV2/Accident/combined_data_in_taiwan.csv'
OUTPUT_PATH = '../ComputedDataV5/ForModel/full_hex_grid.csv'

local_tasks = {
    'count_mrt': '../ComputedData/MRT/full_mrt.csv',
    'count_youbike': '../ComputedData/YouBike/full_youbike.csv',
    'count_parking_official': '../ComputedData/Parkinglot/full_parkinglot.csv'
}

BASE_GRID_PATH = '../ComputedDataV5/ForModel/base_hex_grid.csv'
ANALYSIS_OUTPUT_PATH = '../ComputedDataV5/ForModel/full_hotspots.csv'

In [ ]:
generator = TaiwanBaseGridGenerator(osm_dir=OSM_DIR, boundary_shp_path=SHP_PATH)
generator.generate_grid(radius_meters=100)
generator.calculate_osm_features()
generator.add_local_features(local_tasks)
generator.save_base_grid(BASE_GRID_PATH)

In [ ]:
analyzer = AccidentHotspotAnalyzer(base_grid_path=BASE_GRID_PATH)
analyzer.integrate_accident_data(ACCIDENT_PATH, filter_query=None)
analyzer.calculate_hotspots()
analyzer.save_result(ANALYSIS_OUTPUT_PATH)

In [14]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point, box
from shapely import wkt

# roads_path = '/Users/wangqiqian/Desktop/ST-RTA-GIS/Data/road_new.shp/gis_osm_roads_free_1.shp'
# gdf_roads = gpd.read_file(roads_path)

# all_coords = [pt for line in gdf_roads.geometry for pt in line.coords]

# df_points = pd.DataFrame(all_coords, columns=['x', 'y'])
# point_counts = df_points.value_counts()
# junctions = point_counts[point_counts >= 2].index.tolist()

# gdf_junctions = gpd.GeoDataFrame(
#     geometry=[Point(x, y) for x, y in junctions],
#     crs=gdf_roads.crs
# )

roads_path = '/Users/wangqiqian/Desktop/ST-RTA-GIS/Data/road_new.shp/gis_osm_roads_free_1.shp'
gdf_roads = gpd.read_file(roads_path)
car_roads = ['motorway', 'motorway_link', 'trunk', 'trunk_link', 'primary', 'primary_link', 
             'secondary', 'secondary_link', 'tertiary', 'tertiary_link', 'unclassified', 
             'residential', 'living_street', 'service']
gdf_roads = gdf_roads[gdf_roads['fclass'].isin(car_roads)]

all_coords = [pt for line in gdf_roads.geometry for pt in line.coords]
df_points = pd.DataFrame(all_coords, columns=['x', 'y'])
point_counts = df_points.value_counts()

way = point_counts[point_counts >= 2].index.tolist()
gdf_way = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in way], crs=gdf_roads.crs)

df_grid = pd.read_csv(ANALYSIS_OUTPUT_PATH)
df_grid['geometry'] = df_grid['geometry'].apply(wkt.loads)
gdf_grid = gpd.GeoDataFrame(df_grid, geometry='geometry', crs="EPSG:3826")

if gdf_way.crs != gdf_grid.crs:
    gdf_way = gdf_way.to_crs(gdf_grid.crs)

joined = gpd.sjoin(gdf_way, gdf_grid[['grid_id', 'geometry']], predicate='within')
count = joined.groupby('grid_id').size()
count.name = 'count_intersection'

gdf_grid = gdf_grid.merge(count, on='grid_id', how='left')
gdf_grid['count_intersection'] = gdf_grid['count_intersection'].fillna(0).astype(int)

OUTPUT_PATH = '../ComputedDataV5/ForModel/base_hex_grid_final.csv'
gdf_grid.to_csv(OUTPUT_PATH, index=False)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

def plot_hotspot_map(gdf):
    fig, ax = plt.subplots(figsize=(12, 12))

    color_map = {
        'Hot Spot (99%)': '#800026',
        'Hot Spot (95%)': '#FC4E2A',
        'Hot Spot (90%)': '#FD8D3C',
        'Not Significant': '#d9d9d9',
        'Cold Spot (90%)': '#6baed6',
        'Cold Spot (95%)': '#3182bd',
        'Cold Spot (99%)': '#08519c' 
    }
    
    ns_data = gdf[gdf['gi_category'] == 'Not Significant']
    if not ns_data.empty:
        ns_data.plot(
            ax=ax, 
            color=color_map['Not Significant'], 
            edgecolor='none',
            alpha=0.5
        )

    categories = [
        'Cold Spot (99%)', 'Cold Spot (95%)', 'Cold Spot (90%)',
        'Hot Spot (90%)', 'Hot Spot (95%)', 'Hot Spot (99%)'
    ]
    
    legend_handles = []
    
    for cat in categories:
        subset = gdf[gdf['gi_category'] == cat]
        if not subset.empty:
            subset.plot(
                ax=ax, 
                color=color_map[cat], 
                edgecolor='black',
                linewidth=0.1
            )
            legend_handles.append(mpatches.Patch(color=color_map[cat], label=cat))

    ax.set_title('Getis-Ord Gi* Hot Spot Analysis (Accidents)', fontsize=15, fontweight='bold')
    ax.axis('off')
    if legend_handles:
        ax.legend(handles=legend_handles, loc='lower right', title='Confidence Level', frameon=True)
    
    plt.tight_layout()
    plt.show()

plot_hotspot_map(hex_gi)

## Morans

In [ ]:
import libpysal
from esda.moran import Moran, Moran_Local
import matplotlib.pyplot as plt

def calculate_morans(gdf, target_col='accident_count'):
    work_gdf = gdf.copy()
    work_gdf[target_col] = work_gdf[target_col].fillna(0)
    
    w = libpysal.weights.Queen.from_dataframe(work_gdf, use_index=True)

    if w.islands:
        print(f'found {w.islands} islands, removing from analysis')
        work_gdf = work_gdf.drop(index=w.islands)
        w = libpysal.weights.Queen.from_dataframe(work_gdf, use_index=True)

    w.transform = 'r'
    # Global Moran's I
    y = work_gdf[target_col].values
    moran = Moran(y, w)

    print('global moran result')
    print(f"   Moran's I : {moran.I:.4f}")
    print(f"   P-value : {moran.p_sim:.4f}")

    print('local morans')
    lisa = Moran_Local(y, w)
    
    work_gdf['lisa_q'] = lisa.q
    work_gdf['lisa_p'] = lisa.p_sim
    
    work_gdf['cluster_type'] = 'Not Significant'
    sig = work_gdf['lisa_p'] < 0.05

    work_gdf.loc[sig & (work_gdf['lisa_q']==1), 'cluster_type'] = 'High-High (Hotspot)'
    work_gdf.loc[sig & (work_gdf['lisa_q']==3), 'cluster_type'] = 'Low-Low (Coldspot)'
    work_gdf.loc[sig & (work_gdf['lisa_q']==2), 'cluster_type'] = 'Low-High (Outlier)'
    work_gdf.loc[sig & (work_gdf['lisa_q']==4), 'cluster_type'] = 'High-Low (Outlier)'
    
    return work_gdf, moran

hex_lisa, global_moran = calculate_morans(hex_grid_final, 'accident_count')

In [ ]:
def plot_lisa_map(gdf):
    fig, ax = plt.subplots(figsize=(12, 12))

    color_map = {
        'High-High (Hotspot)': '#d7191c',
        'Low-Low (Coldspot)': '#2c7bb6',
        'Low-High (Outlier)': '#83b9e2',
        'High-Low (Outlier)': '#fdae61',
        'Not Significant': '#eeeeee'
    }

    gdf[gdf['cluster_type'] == 'Not Significant'].plot(
        ax=ax, color='#eeeeee', edgecolor='none', alpha=0.5
    )

    for ctype, color in color_map.items():
        if ctype == 'Not Significant': continue
        subset = gdf[gdf['cluster_type'] == ctype]
        if len(subset) > 0:
            subset.plot(
                ax=ax, 
                color=color, 
                edgecolor='black', 
                linewidth=0.1, 
                label=ctype
            )
    
    plt.title('LISA Cluster Map of Accidents (Moran\'s I)', fontsize=15)
    plt.legend(loc='lower right')
    plt.axis('off')
    plt.show()

plot_lisa_map(hex_lisa)